In [18]:
# pyright: reportMissingImports=false
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F

from src.shared.gaussian import Gaussian
from src.shared.faces import make_cube_faces
from src.shared.sides import (    
    to_vertices,
    make_phi_theta,
    sphered_vertices,
    to_spherical,
    set_edges,
    set_corners,
)

import torch
from pytorch_wavelets import DWTForward, DWTInverse

class WaveGenerator(nn.Module):
    def __init__(self, n, kernel=3, sigma=1, padding=1, r=0.5):
        super(WaveGenerator, self).__init__()        
        self.n = n
        
        self.register_buffer('faces', make_cube_faces(n))        
        self.register_buffer('coarse', sphered_vertices(6, r))
                
        self.lvl1 = nn.Parameter(torch.zeros(6, 3, 3, 9, 9))        
        self.lvl2 = nn.Parameter(torch.zeros(6, 3, 3, 6, 6))

        self.gaussian =  Gaussian(kernel, sigma=sigma, padding=padding)
        
        self.wave = 'db2'
        self.mode = 'symmetric'
        self.levels = 2
        self.xfm  = DWTForward(J=self.levels, mode=self.mode, wave=self.wave)
        self.ifm = DWTInverse(mode=self.mode, wave=self.wave) 
       
    def forward(self): 
        Yl = self.coarse
        Yh = [self.lvl1, self.lvl2]
        stacked = self.ifm((Yl, Yh))        
        vert = to_vertices(stacked)
        return vert, self.faces, stacked

G = WaveGenerator(16).cuda()   
v, f, stacked = G()
[p.shape for p in [v, f, stacked]]

[torch.Size([1536, 3]), torch.Size([3068, 3]), torch.Size([6, 3, 16, 16])]

In [13]:
self.coarse.shape

torch.Size([6, 3, 16, 16])

In [7]:
self = G
Yl = self.coarse
Yh = [self.lvl1, self.lvl2]
stacked = self.ifm((Yl, Yh))

RuntimeError: The size of tensor a (15) must match the size of tensor b (6) at non-singleton dimension 3

In [9]:
self.lvl1.shape, self.lvl2.shape

(torch.Size([6, 3, 3, 9, 9]), torch.Size([6, 3, 3, 6, 6]))